### Puzzle

https://adventofcode.com/2020/day/19

### Imports

In [342]:
import regex

### Load Input

In [547]:
# Store the location of the input directory
data_dir = '../../../data/2020'

# Open the input and store a list of each item as an int
with open(f"{data_dir}/day19_input.txt") as f:
    inputs = f.read().splitlines()

### Part 1

In [548]:
def replace_next_key(rule, rules_dict):
    # Initialize a pair of indices that will be used to extract a number with any number of digits
    start_index = 0
    end_index = -1

    # Traverse the rule
    for index, char in enumerate(rule):
        next_index = index

        # Make sure the second digit of a number isn't being recounted
        if index > end_index:
            
            # Get all digits of the next number
            while rule[next_index].isdigit():
                start_index = index
                next_index += 1
            end_index = next_index
            
            # Store the number and replace that number in the current rule with that number's rules
            key = rule[start_index:end_index]
            if key.isdigit():
                rule = rule[:start_index] + rules_dict[key] + rule[end_index:]
                return rule

In [549]:
# Split input into rules and messages based on the empty line
rules = inputs[:inputs.index('')]
messages = inputs[inputs.index('') + 1:]

In [550]:
# Initialize a dictionary to store all of the rule mappings
rules_dict = dict()

# Clean up the rules so that parentheses, pipes, and quotes are appropriately handled 
for rule in rules:
    rule_number = rule.split(':')[0]
    rule_info = rule.split(':')[1].strip().replace('"', '').split(' ')
    rule_info = ''.join([f"({char})" if char.isdigit() else char for char in rule_info])
    
    rules_dict[rule_number] = rule_info

In [513]:
# Replace all numbered rules until only letters remain
while sum([char.isdigit() for char in rules_dict['0']]) > 0:
    rules_dict['0'] = replace_next_key(rules_dict['0'], rules_dict)

# Initialize a counter to count valid messages
valid_messages = 0

# Use Python's regular expression package to check if a message is valid and if so, add it to the counter
for message in messages:
    if regex.fullmatch(rules_dict['0'].replace(' ', ''), message):
        valid_messages += 1

In [514]:
valid_messages

3

### Part 2

In [515]:
def replace_next_key_v2(rule, rules_dict):
    # Initialize a pair of indices that will be used to extract a number with any number of digits
    start_index = 0
    end_index = -1

    # Traverse the rule
    for index, char in enumerate(rule):
        next_index = index

        # Make sure the second digit of a number isn't being recounted
        if index > end_index:
            
            # Get all digits of the next number
            while rule[next_index].isdigit():
                start_index = index
                next_index += 1
            end_index = next_index
            
            # Store the number and replace that number in the current rule with that number's rules
            key = rule[start_index:end_index]
            if key.isdigit():
                if key == '8':
                    rule = rule[:start_index] + '(42)+' + rule[end_index:]
                elif key == '11':
                    rule = rule[:start_index] + '(42)+(31)+' + rule[end_index:]
                else:
                    rule = rule[:start_index] + rules_dict[key] + rule[end_index:]
                return rule

In [551]:
# Update rules 8 and 11, according to the directions
rules_dict['8'] = '(42) | (42) (8)'
rules_dict['11'] = '(42) (31) | (42) (11) (31)'

In [552]:
# Replace all numbered rules until only letters remain
while sum([char.isdigit() for char in rules_dict['0']]) > 0:
    rules_dict['0'] = replace_next_key_v2(rules_dict['0'], rules_dict)

In [553]:
# Do the same for just the 42-chain
while sum([char.isdigit() for char in rules_dict['42']]) > 0:
    rules_dict['42'] = replace_next_key_v2(rules_dict['42'], rules_dict)

In [554]:
# Initialize a list to store valid messages plus some that are invalid but will be checked shortly
valid_messages_temp = []

# Use Python's regular expression package to check if a message is valid and if so, add it to the counter
for message in messages:
    if re.fullmatch(rules_dict['0'].replace(' ', ''), message):
        valid_messages_temp.append(message)

In [556]:
# The only key that uses the infinite loops is 0: 8 11 which becomes 0: 42 42 31
# The messages from keys 42 and 31 are both 8-digit messages
# We can individually verify every 8-digit sequence if it is a code-42 sequence or a code-31 sequence
# There must more code-42 sequences than code-31 sequences
# If the middle chain is a code-42 sequence, then it must be a valid message

# Initialize a counter to count which of the temporary valid messages pass the additional check
valid_messages = 0

# Only check the messages that passed the first test
for valid_message_temp in valid_messages_temp:
    
    # Get the middle 8-digit sequence
    n_sequences = len(valid_message_temp)/8
    if n_sequences%2 == 0:
        middle_sequence_start = (n_sequences/2)*8
        middle_sequence_end = (n_sequences/2)*8 + 8
    else:
        middle_sequence_start = ((n_sequences/2)-0.5)*8
        middle_sequence_end = ((n_sequences/2)-0.5)*8 + 8
        
    # Check if the middle 8-digit sequence is a code-42 sequence and if so add it to the counter
    if re.fullmatch(rules_dict['42'].replace(' ', ''), valid_message_temp[int(middle_sequence_start): int(middle_sequence_end)]):
        valid_messages += 1

In [560]:
valid_messages

350